In [ ]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv
from openai import OpenAI

_ = load_dotenv()

client = OpenAI()

prompt = """
You run in a loop of Thought, Suggestions ,Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the suitable actions available to you - then return
PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use
floating point syntax if necessary

get_average_price:
e.g. get_average_price: Lenovo
returns average price of the laptop when given the model name

Example session:

Question: How much does a Lenovo Laptop costs?
Thought: I should look the Laptop price using get_average_price

Action: get_average_price: Lenovo
PAUSE

You will be called again with this:

Observation: A lenovo laptop average price is $400

You then output:

Answer: A lenovo laptop costs $400
""".strip()

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

def calculate(what):
    return eval(what)

def get_average_price(name):
    if name in "Lenovo":
        return("a lenovo laptop average price is $400")
    elif name in "Dell":
        return("a Dell laptop average price is $450.")
    elif name in "Asus":
        return("a Asus laptop average price is $500")
    else:
        return("An laptop average price is $430")

known_tools = {
    "calculate": calculate,
    "get_average_price": get_average_price
}

action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            print(actions)
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in Tools:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = Tools[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

question = """what is the total cost for 2 lenovo and a Asus laptops"""
query(question)